In [1]:
%reload_ext autoreload
%autoreload 2
from EnzymePynetics.core import Estimator

In [2]:
estimator, enzymeml = Estimator.from_enzymeml(
    enzymeml="simulated_enzymeML.json",
    measured_reactant="substrate",
)

In [3]:
substrate, product = estimator.reactants
enzyme = estimator.enzymes[0]
substrate

Reactant(id='reactant8', name='substrate', vessel_id='vessel4', init_conc=None, constant=False, unit=None, uri=None, creator_id=None, smiles=None, inchi=None, chebi_id=None, ontology='SBO:0000247')

In [4]:
# Define Reaction
oxidation = estimator.add_reaction(
    id="r1", name="Oxidation", educt="reactant8", product=product, catalyst=enzyme
)

In [5]:
# Define potential substrate rate-laws
michaelis = estimator.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * catalyst * k_cat / (K_M + substrate)",
)

competitive_product = estimator.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * catalyst * k_cat / (K_M * (1 + product / K_ic) + substrate)",
)

substrate_inhibition = estimator.add_model(
    id="model4",
    name="substrate inhibition",
    equation="substrate = -k_cat * catalyst * substrate / (K_M + ((1+(substrate/K_iu))*substrate))",
)

# Define potential enzyme rate-lawscatalyst
enzyme_inactivation = estimator.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="catalyst = -k_ie * catalyst",
)

In [6]:
estimator.models[1].eq_species

{'catalyst', 'product', 'substrate'}

In [7]:
estimator.fit_models()

Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:01<00:00,  5.29it/s]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,mmol / l,1 / min / mmol / l,mmol / l,mmol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-3572,12.000 ± 0.00 %,17.000 ± 0.00 %,,10.000 ± 0.00 %,,0.100 ± 0.00 %
michaelis-menten with enzyme inactivation,-133,11.600 ± 1.44 %,21.060 ± 3.04 %,,,,0.197 ± 1.92 %
competitive product inhibition,-121,14.025 ± 4.25 %,11.943 ± 15.13 %,,2.219 ± 14.03 %,,
substrate inhibition with enzyme inactivation,-82,16.631 ± 5.08 %,35.375 ± 6.57 %,,,150.000 ± 34.05 %,0.199 ± 3.03 %
michaelis-menten,117,7.810 ± 13.75 %,33.515 ± 27.68 %,,,,
substrate inhibition,121,11.458 ± > 100 %,56.022 ± > 100 %,,,150.000 ± > 100 %,


In [8]:
estimator.visualize()
estimator.correlations()

In [9]:
# Get the best model system
model_system = estimator.get_reaction_system(
    "competitive product inhibition with enzyme inactivation"
)

# Safe the model system to the EnzymeML document
enzymeml_with_kinetic_model = estimator.to_enzymeml(enzymeml, model_system)

In [10]:
print(model_system)

ReactionSystem
├── id = reactionsystem3
├── name = competitive product inhibition with enzyme inactivation
├── reactions
│   ├── 0
│   │   └── Reaction
│   │       ├── id = r1
│   │       ├── name = Oxidation
│   │       ├── reversible = False
│   │       ├── temperature = 30.0
│   │       ├── temperature_unit = C
│   │       ├── ph = 4.0
│   │       ├── ontology = SBO:0000176
│   │       ├── model
│   │       │   └── KineticModel
│   │       │       ├── id = model3
│   │       │       ├── name = competitive product inhibition
│   │       │       ├── equation =  -reactant8 * protein4 * k_cat / (K_M * (1 + reactant9 / K_ic) + reactant8)
│   │       │       └── parameters
│   │       │           ├── 0
│   │       │           │   └── KineticParameter
│   │       │           │       ├── id = kineticparameter2
│   │       │           │       ├── name = K_M
│   │       │           │       ├── value = 17.00000000000007
│   │       │           │       ├── unit = mmol / l
│   │       │         

In [11]:
for reaction in enzymeml_with_kinetic_model.reactions:
    print(reaction)

Reaction
├── id = r1
├── name = Oxidation
├── reversible = False
├── temperature = 30.0
├── temperature_unit = C
├── ph = 4.0
├── ontology = SBO:0000176
├── model
│   └── KineticModel
│       ├── id = model3
│       ├── name = competitive product inhibition
│       ├── equation = substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)
│       └── parameters
│           ├── 0
│           │   └── KineticParameter
│           │       ├── id = kineticparameter0
│           │       ├── name = K_ic
│           │       ├── value = 2.219870043199241
│           │       ├── unit = mmol / l
│           │       ├── initial_value = 3.0
│           │       ├── upper = 3000.0
│           │       ├── lower = 0.003
│           │       ├── is_global = False
│           │       ├── stdev = 0.31122332440683753
│           │       └── constant = False
│           ├── 1
│           │   └── KineticParameter
│           │       ├── id = kineticparameter1
│           │       ├── nam